In [ ]:
import boto3

In [ ]:
emr_client = boto3.client('emr')

In [ ]:
emr_client.list_clusters(ClusterStates=['STARTING', 'RUNNING', 'WAITING'])

In [ ]:
step_details = emr_client.add_job_flow_steps(
    JobFlowId='j-Y4YGUCT4IH73',
    Steps=[
        {
            'Name': 'Compute Daily Revenue using Spark SQL',
            'HadoopJarStep': {
                'Jar': 'command-runner.jar',
                'Args': [
                    'spark-sql',
                    '-d',
                    's3.bucket=airetail',
                    '-f',
                    's3://airetail/scripts/08_compute_daily_product_revenue_args.sql'
                ]
            }
        }
    ]
)

In [ ]:
step_details['StepIds'][0]

In [ ]:
step_waiter = emr_client.get_waiter("step_complete")

print(f'''Waiting for job step ({step_details['StepIds'][0]}) to complete''')
step_waiter.wait(
    ClusterId='j-Y4YGUCT4IH73',
    StepId=step_details['StepIds'][0],
    WaiterConfig={
        "Delay": 30,
        "MaxAttempts": 10
    }
)

print(f'''Step ({step_details['StepIds'][0]}) is completed''')